# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Moses Koroma

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [2]:
import requests
import json

# Step 1: Call the API
response = requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul')

# Step 2: Convert response to JSON
result = response.json()

# Step 3: Save JSON to a local file
with open('birdhouse_lyrics.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, indent=4)

print("Lyrics saved to birdhouse_lyrics.json")


Lyrics saved to birdhouse_lyrics.json


2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [4]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from spacy.tokens import Doc
import json

# Load model and add sentiment pipe
nlp = spacy.load("en_core_web_sm")
if "spacytextblob" not in nlp.pipe_names:
    nlp.add_pipe("spacytextblob")

# Manually register polarity extension (fixes the error)
if not Doc.has_extension("polarity"):
    Doc.set_extension("polarity", getter=lambda doc: doc._.blob.polarity)

# Load lyrics
with open("birdhouse_lyrics.json", "r", encoding="utf-8") as f:
    data = json.load(f)

lyrics = data["lyrics"]
print("Lyrics Preview:\n")
print(lyrics[:500])

# Run sentiment analysis
doc = nlp(lyrics)
polarity = doc._.polarity

print("\nPolarity Score:", polarity)

Lyrics Preview:

I'm your only friend 
I'm not your only friend 
But I'm a little glowing friend 
But really I'm not actually your friend 
But I am 


Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 

Say I'm the only bee in your bonnet 

Make a little birdhouse in your soul 



I have a secret to tell 

From my electrical well 

It's a simple message and I'm leaving out the whistles and bells 

So the room must liste

Polarity Score: 0.04505208333333333


3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [ ]:
import requests
import json

def save_lyrics(artist, song, filename):
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    response = requests.get(url)

    if response.status_code == 200 and response.text.strip():
        try:
            data = response.json()
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(data, f)
            print(f"Lyrics saved to {filename}")
        except json.JSONDecodeError:
            print(f" Could not decode JSON for {artist} - {song}")
    else:
        print(f" Failed to get lyrics for {artist} - {song}")

# Test with 4 known, simple songs
save_lyrics("Adele", "Hello", "hello.json")
save_lyrics("Coldplay", "Yellow", "yellow.json")



Lyrics saved to hello.json
Lyrics saved to yellow.json
⚠️ Failed to get lyrics for Drake - Gods Plan
⚠️ Failed to get lyrics for Eminem - Lose Yourself


4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.